In [ ]:
!pip install simpletransformers
#!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 2

In [ ]:
# Load drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Needed only for XLM model
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Importing Necessary Libraries

In [ ]:
# Libraries
#import sacremoses
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel,NERArgs
#from torch.utils.tensorboard import SummaryWriter
import pandas as pd

#### Loading dataset

In [ ]:
path = 'path_to_dataset'
data = pd.read_csv(path,encoding="utf-8" )

In [ ]:
data.head(5)

,sentence#,word,pos,ner
0,sentence: 1,0,CD,I-MISC
1,NaN,पृष्ठभूमि,NN,O
2,NaN,:,YM,O
3,NaN,विश्व,NN,O
4,NaN,का,PKO,O


In [ ]:
data =data.fillna(method ="ffill")

In [ ]:
data.head(5)

,sentence#,word,pos,ner
0,sentence: 1,0,CD,I-MISC
1,sentence: 1,पृष्ठभूमि,NN,O
2,sentence: 1,:,YM,O
3,sentence: 1,विश्व,NN,O
4,sentence: 1,का,PKO,O


#### Encoding sentence identifiers

In [ ]:
data["sentence#"] = LabelEncoder().fit_transform(data["sentence#"] )

In [ ]:
data.head(5)

,sentence#,word,pos,ner
0,0,0,CD,I-MISC
1,0,पृष्ठभूमि,NN,O
2,0,:,YM,O
3,0,विश्व,NN,O
4,0,का,PKO,O


#### Renaming columns name to labels

In [ ]:
#Note that 'pos' should be used instead of 'ner' for POS tagging task
data.rename(columns={"sentence#":"sentence_id","word":"words","ner":"labels"}, inplace =True)

In [ ]:
data.tail(5)

,sentence_id,words,pos,labels
13829,406,मा,PP,O
13830,406,समानता,NN,O
13831,406,छैन,VF,O
13832,406,भन्नुभयो,VF,O
13833,406,।,YF,O


In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

####Building train and test data

In [ ]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})
print(len(train_data),len(test_data))

11067 2767


In [ ]:
train_data

,sentence_id,words,labels
12362,362,को,O
1327,233,",",O
13776,406,लाई,O
3451,27,जस,O
9662,261,एक,B-MISC
...,...,...,...
2427,448,थियो,O
4205,31,पनि,O
10759,311,सके,O
3824,30,विचार,O


#### Model Fine-tuning

In [ ]:
label = data["labels"].unique().tolist()
label

['I-MISC',
 'O',
 'B-LOC',
 'B-MISC',
 'I-LOC',
 'B-ORG',
 'I-ORG',
 'B-PER',
 'I-PER',
 'E-MISC',
 'V']

In [ ]:
# Model arguments and Parameters
args = NERArgs()
args.num_train_epochs = 5
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 8
args.eval_batch_size = 8
args.save_model_every_epoch=False

In [ ]:
# Change according to model eg. 'xlmroberta', 'xlm-roberta-base'. Refer to simpletransformer docs and hugging face for more models
model = NERModel('bert', 'NepBERTa/NepBERTa',labels=label,args =args,use_cuda=True,from_tf=True)# remove 'from_tf=True' for other models

All TF 2.0 model weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.


In [ ]:
# Training the model
model.train_model(train_data,eval_data=test_data, acc=accuracy_score, verbose=True)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/59 [00:00<?, ?it/s]

(295, 0.20605409672736363)

In [ ]:
# Evaluating scores
result, model_outputs, preds_list = model.eval_model(test_data, verbose=True)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

In [ ]:
# Final result
result

{'eval_loss': 0.1278538624919428,
 'precision': 0.8385416666666666,
 'recall': 0.7419354838709677,
 'f1_score': 0.787286063569682}